In [13]:
import requests
import pendulum
from pathlib import Path
import os
import sys

In [15]:
"""
Caso não exista a pasta 'downloads', ela é criada ao chamar este módulo
"""
#caminho_base = utils.dir_abs
diretorio_saida = Path(f'cfs_clima_91_20/')
diretorio_saida.mkdir(exist_ok=True, parents=True)


def auxiliar_download(url_base:str, data_requerida:pendulum.datetime, nome_arquivo:str) -> bool:
    """ Subfunção de download(). Não é chamada diretamente.
    Faz a requisição do dado de TSM no repositório da NOAA.
    Args:
        url_base (str): Parte imutável do link de endereço.
        data_requerida (pendulum.datetime): data do dado procurado.
        nome_arquivo (str): [description] Nome do arquivo a ser requisitado.
    Returns:
        bool: True = dado baixado, False= erro 404 | arquivo não disponível.
    """

    url=f'{url_base}/{nome_arquivo}'
    resp=requests.get(url)

    if resp.status_code == 200:
        resp=requests.get(url).content
        diretorio = Path(diretorio_saida, nome_arquivo)

        with open(diretorio, "wb") as arquivo_:
            arquivo_.write(resp)
            print(f"{nome_arquivo} [ok]")
            mensagem_de_sucesso = True

    elif resp.status_code == 404:
        print(f'Erro 404. {nome_arquivo} ainda não disponível')
        mensagem_de_sucesso = False

    return mensagem_de_sucesso


def download(data_string:str=None, hoje:bool=False) -> str:
    """Chama a função de download dos dados de temperatura oceânica do NOAA.
    Caso o dado final não esteja pronto, baixa o preliminar.
    Args:
        data_string (str, optional): [DD-MM-YYYY] data do dado procurado. Defaults to None.
        hoje (bool, optional): Habilita o dia atual na hora da execução. Sobrescreve o data_string. Defaults to False.
    Returns:
        str: Nome do arquivo baixado
    """

    data_requerida = pendulum.now('America/Sao_Paulo') if hoje else pendulum.from_format(data_string, 'DD-MM-YYYY')
    #url_base = 'https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr'
    url_base2='https://ftp.emc.ncep.noaa.gov/cmb/sst/oimonth_v2/GRIB'
    url_base='https://ftp.cpc.ncep.noaa.gov/precip/PORT/sst/oimonth_v2/GRIB/'

    nome_arquivo = f'oiv2mon.{data_requerida.format("YYYYMM")}.grb'	
    sucesso = auxiliar_download(url_base, data_requerida, nome_arquivo)
    if sucesso:
        print(f'{nome_arquivo} {sucesso}')
    else:
        print(f'>>> Tratativa de erro - link 2<<< {nome_arquivo}')
        sucesso = auxiliar_download(url_base2, data_requerida, nome_arquivo)
    '''
    if sucesso:
        tipo = "consistido"
        return nome_arquivo, tipo
    else :
        nome_arquivo =  f'oiv2mon.{data_requerida.format("YYYYMM")}198111.grb'	
        sucesso = auxiliar_download(url_base, data_requerida, nome_arquivo)
        
        if sucesso:
            tipo = "preliminar"
            return nome_arquivo, tipo
        else:
            sys.exit('A data solicitada ainda não possui dados')
    '''

In [ ]:
#import tsm
import pendulum

inicio = pendulum.datetime(1991,1,1)    #(1991, 1, 1)
fim = pendulum.datetime(2020, 12, 31)
periodo = pendulum.period(inicio, fim)

for dt in periodo.range('months'):
    data_string = dt.format('DD-MM-YYYY')

    download(data_string=data_string)

In [17]:
import xarray as xr

In [56]:
#ds = xr.open_dataset('download/oiv2mon.202012.grb')

Ignoring index file 'download/oiv2mon.202012.grb.923a8.idx' older than GRIB file


In [18]:
diretorio_saida

PosixPath('cfs_clima_91_20')

In [19]:
dataset = xr.open_mfdataset(f'{diretorio_saida}/*.grb', concat_dim="valid_time", combine="nested",
                data_vars='minimal', coords='minimal', compat='override',
                backend_kwargs={
                    "filter_by_keys": {
                        "typeOfLevel": "surface",
                        "shortName": "t",
                    },
                 },
                parallel=True)

In [44]:
type(i.values)

numpy.datetime64

In [50]:
dataset.sel({"valid_time":[i.values]})

<xarray.Dataset>
Dimensions:     (latitude: 180, longitude: 360, valid_time: 1)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
  * longitude   (longitude) float64 -0.5 0.5 1.5 2.5 ... 355.5 356.5 357.5 358.5
  * valid_time  (valid_time) datetime64[ns] 2022-08-01
Data variables:
    t           (valid_time, latitude, longitude) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
Attributes:
    GRIB_edition:            1
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          4
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2022-09-23T16:46 GRIB to CDM+CF via cfgrib-0.9.1...

In [20]:
# para criar climatologia
clima_tsm = dataset.t.groupby('valid_time.month').mean()

# esse aqui foi para criar histórico completo


In [21]:
clima_tsm

<xarray.DataArray 't' (month: 12, latitude: 180, longitude: 360)>
dask.array<stack, shape=(12, 180, 360), dtype=float32, chunksize=(1, 180, 360), chunktype=numpy.ndarray>
Coordinates:
    time       datetime64[ns] ...
    step       timedelta64[ns] ...
    surface    float64 ...
  * latitude   (latitude) float64 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
  * longitude  (longitude) float64 -0.5 0.5 1.5 2.5 ... 355.5 356.5 357.5 358.5
  * month      (month) int64 1 2 3 4 5 6 7 8 9 10 11 12

In [36]:
clima_tsm.to_netcdf('oisst_climatologia_1982-2009.nc')

In [22]:
import recorta_dado
import pandas as pd
from regioes_indices import regioes
import xarray as xr
import pendulum

In [46]:
dicionario = {
    'mes': [],
    "TSM Niño 1+2" : [],
    "TSM Niño 3" : [],
    "TSM Niño 3.4" : [],
    "TSM Niño 4" : [],
    "TSM CBM" : [],
    "TSM TNA" : [],
    "TSM TSA" : [],
}



In [37]:
from utils import padronizar_longitude
from utils import delimitar_regiao
from utils import converter_para_celsius

In [62]:
clima_tsm_ = clima_tsm - 273.15

In [ ]:
for mes in range(1,12+1):
    tsm = clima_tsm_.sel(month=mes)
    dicionario['mes'].append(mes)
    
    for indice in regioes:
        
        '''
        tsm_nino = recorta_dado.main(tsm,
            contorno_tipo='lat_lon',
            tipo='grib',
            latitude=[ indice.latitude[0], indice.latitude[1] ], 
            longitude=[ indice.longitude[0], indice.longitude[1] ]
            )
        '''
        tsm = padronizar_longitude(tsm, xdim='longitude')
        valor = delimitar_regiao(obj=tsm, indice=indice)
        
        #valor = tsm_nino.mean(dim=["latitude", "longitude"]) 
        #valor = valor - 273.15
        valor = round(float(valor.values), 2)

        dicionario[indice.nome].append(valor)

        print(f'O valor da região {indice.nome} no mês {mes} é : {valor}')

In [64]:
dt = pd.DataFrame.from_dict(dicionario)

In [33]:
dt.set_index('mes').to_csv('oiv2_clima_1991-2020.csv', sep=';')

In [79]:
tsm

<xarray.DataArray 't' (valid_time: 1, latitude: 180, longitude: 360)>
array([[[271.35, 271.35, ..., 271.35, 271.35],
        [271.35, 271.35, ..., 271.35, 271.35],
        ...,
        [271.35, 271.35, ..., 271.35, 271.35],
        [271.35, 271.35, ..., 271.35, 271.35]]], dtype=float32)
Coordinates:
    time        datetime64[ns] 1981-11-01
    step        timedelta64[ns] 00:00:00
    surface     float64 0.0
  * latitude    (latitude) float64 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
  * longitude   (longitude) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * valid_time  (valid_time) datetime64[ns] 2022-08-01
Attributes:
    GRIB_paramId:                             130
    GRIB_numberOfPoints:                      64800
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            avgua
    GRIB_gridType:                            regular_ll
    GRIB_NV:                                  0
    GRIB_Nx:                                  360
    GRIB_Ny:                                  180
    GRIB_cfName:                              air_temperature
    GRIB_cfVarName:                           t
    GRIB_gridDefinitionDescription:           Latitude/Longitude Grid
    GRIB_iDirectionIncrementInDegrees:        1.0
    GRIB_iScansNegatively:                    0
    GRIB_jDirectionIncrementInDegrees:        1.0
    GRIB_jPointsAreConsecutive:               0
    GRIB_jScansPositively:                    0
    GRIB_latitudeOfFirstGridPointInDegrees:   89.5
    GRIB_latitudeOfLastGridPointInDegrees:    -89.5
    GRIB_longitudeOfFirstGridPointInDegrees:  0.5
    GRIB_longitudeOfLastGridPointInDegrees:   -0.5
    GRIB_missingValue:                        9999
    GRIB_name:                                Temperature
    GRIB_shortName:                           t
    GRIB_units:                               K
    long_name:                                Temperature
    units:                                    K
    standard_name:                            air_temperature

In [68]:
clima_oiv2 = pd.read_csv('oiv2_clima_1991-2020.csv', sep=';', index_col='mes')

In [11]:
clima_texto = "1991-2020 OIv2.1"
clima_oiv2_dict = {'mes': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 'nino34': [26.51,
  26.7,
  27.26,
  27.87,
  27.92,
  27.72,
  27.27,
  26.8,
  26.64,
  26.65,
  26.68,
  26.58]}
clima_oiv2 = pd.DataFrame.from_dict(clima_oiv2_dict).set_index('mes')

In [69]:
dicionario2 = {
    'data'   : [],
    "TSM Niño 1+2" : [],
    "TSM Niño 1+2 anom" : [],
    "TSM Niño 3" : [],
    "TSM Niño 3 anom" : [],
    "TSM Niño 3.4" : [],
    "TSM Niño 3.4 anom" : [],
    "TSM Niño 4" : [],
    "TSM Niño 4 anom" : [],
    "TSM CBM" : [],
    "TSM CBM anom" : [],
    "TSM TNA" : [],
    "TSM TNA anom" : [],
    "TSM TSA" : [],
    "TSM TSA anom" : [],
}

In [70]:
dataset = xr.open_dataset('oiv2_historico_1981-2022.nc')

In [74]:
dicionario2 = {
    'data'   : [],
    "TSM Niño 1+2" : [],
    "TSM Niño 1+2 anom" : [],
    "TSM Niño 3" : [],
    "TSM Niño 3 anom" : [],
    "TSM Niño 3.4" : [],
    "TSM Niño 3.4 anom" : [],
    "TSM Niño 4" : [],
    "TSM Niño 4 anom" : [],
    "TSM CBM" : [],
    "TSM CBM anom" : [],
    "TSM TNA" : [],
    "TSM TNA anom" : [],
    "TSM TSA" : [],
    "TSM TSA anom" : [],
}

for i in dataset.valid_time:
    
    ds = dataset.sel({"valid_time":[i.values]})
    
    data_str = str(i.values)[:10]
    data = pendulum.from_format(data_str, "YYYY-MM-DD")
    dicionario2['data'].append(data.format('YYYY-MM-DD'))
    
    for indice in regioes:
        
        '''
        tsm_nino = recorta_dado.main(ds.t,
            contorno_tipo='lat_lon',
            tipo='grib',
            latitude=[ indice.latitude[0], indice.latitude[1] ], 
            longitude=[ indice.longitude[0], indice.longitude[1] ]
            )

        valor = tsm_nino.mean(dim=["latitude", "longitude"])
        '''
        tsm = padronizar_longitude(ds.t, xdim='longitude')
        valor = delimitar_regiao(obj=tsm, indice=indice)
        valor = valor - 273.15
        valor = round(float(valor.values), 2)
        mes_int = int(data.format("MM"))
        anomalia = valor - clima_oiv2[indice.nome][mes_int]

        #dicionario[indice.nome].append(valor)
        dicionario2[indice.nome].append(valor)
        dicionario2[f"{indice.nome} anom"].append(anomalia)
        
        #print(f'O valor da região {indice.nome} em {data.format("DD-MM-YYYY")} é : {valor}')

In [78]:
tsm

<xarray.DataArray 't' (valid_time: 1, latitude: 180, longitude: 360)>
array([[[271.35, 271.35, ..., 271.35, 271.35],
        [271.35, 271.35, ..., 271.35, 271.35],
        ...,
        [271.35, 271.35, ..., 271.35, 271.35],
        [271.35, 271.35, ..., 271.35, 271.35]]], dtype=float32)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
  * longitude   (longitude) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * valid_time  (valid_time) datetime64[ns] 2022-08-01
Attributes:
    GRIB_paramId:                             130
    GRIB_numberOfPoints:                      64800
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            avgua
    GRIB_gridType:                            regular_ll
    GRIB_NV:                                  0
    GRIB_Nx:                                  360
    GRIB_Ny:                                  180
    GRIB_cfName:                              air_temperature
    GRIB_cfVarName:                           t
    GRIB_gridDefinitionDescription:           Latitude/Longitude Grid
    GRIB_iDirectionIncrementInDegrees:        1.0
    GRIB_iScansNegatively:                    0
    GRIB_jDirectionIncrementInDegrees:        1.0
    GRIB_jPointsAreConsecutive:               0
    GRIB_jScansPositively:                    0
    GRIB_latitudeOfFirstGridPointInDegrees:   89.5
    GRIB_latitudeOfLastGridPointInDegrees:    -89.5
    GRIB_longitudeOfFirstGridPointInDegrees:  0.5
    GRIB_longitudeOfLastGridPointInDegrees:   -0.5
    GRIB_missingValue:                        9999
    GRIB_name:                                Temperature
    GRIB_shortName:                           t
    GRIB_units:                               K
    long_name:                                Temperature
    units:                                    K
    standard_name:                            air_temperature

In [ ]:
dicionario2

In [75]:
dt = pd.DataFrame.from_dict(dicionario2)

In [76]:
dt

,data,TSM Niño 1+2,TSM Niño 1+2 anom,TSM Niño 3,TSM Niño 3 anom,TSM Niño 3.4,TSM Niño 3.4 anom,TSM Niño 4,TSM Niño 4 anom,TSM CBM,TSM CBM anom,TSM TNA,TSM TNA anom,TSM TSA,TSM TSA anom
0,1981-11-01,21.10,-0.31,26.80,-0.79,25.99,-0.69,27.08,-0.32,15.43,-0.59,26.78,-0.19,24.00,0.09
1,1981-12-01,22.45,-0.13,27.49,-0.17,26.20,-0.38,27.24,-0.23,17.36,-0.81,25.96,0.03,24.66,-0.04
2,1982-01-01,24.01,-0.28,27.09,-0.47,26.68,0.17,27.37,-0.18,19.07,-1.02,24.81,-0.05,25.39,-0.27
3,1982-02-01,25.13,-0.75,26.95,-0.46,26.67,-0.03,27.63,-0.21,19.72,-1.04,23.98,-0.30,26.22,-0.36
4,1982-03-01,24.71,-1.51,26.77,-0.59,27.18,-0.08,27.98,-0.33,20.39,0.18,23.97,-0.25,26.75,-0.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,2022-04-01,23.78,-1.30,27.48,0.00,26.73,-1.14,28.51,-0.10,17.67,-0.73,24.41,-0.11,27.15,0.09
486,2022-05-01,22.24,-1.74,27.67,-0.01,26.71,-1.21,28.18,-0.17,15.80,-0.41,25.17,0.06,26.23,0.00
487,2022-06-01,21.26,-1.38,27.58,-0.09,26.89,-0.83,27.66,-0.02,13.33,-0.77,25.91,0.11,25.15,0.26
488,2022-07-01,20.45,-0.98,27.50,0.06,26.53,-0.74,26.99,-0.01,11.91,-0.69,26.24,-0.16,23.68,0.01


In [82]:
dataset.to_netcdf('oiv2_historico_1981-2022.nc')

In [77]:
dt.set_index('data').to_csv('oiv2_historico_1981-2022.csv', sep=';')